In [ ]:
import numpy as np
import os
import wfdb
from collections import Counter
import pickle
import random
import sys
from tqdm import tqdm

label_group_map = {'N':'N', 'L':'N', 'R':'N', 'V':'V', '/':'Q', 'A':'S', 'F':'F', 'f':'Q', 'j':'S', 'a':'S', 'E':'V', 'J':'S', 'e':'S', 'Q':'Q', 'S':'S'}

def resample_unequal(ts, fs_in, fs_out):
    """
    interploration
    """
    fs_in, fs_out = int(fs_in), int(fs_out)
    if fs_out == fs_in:
        return ts
    else:
        x_old = np.linspace(0, 1, num=fs_in, endpoint=True)
        x_new = np.linspace(0, 1, num=fs_out, endpoint=True)
        y_old = ts
        f = interp1d(x_old, y_old, kind='linear')
        y_new = f(x_new)
        return y_new

if __name__ == "__main__":

    path = 'mit-bih-arrhythmia-database-1.0.0'
    save_path = 'data/'
    # valid_lead = ['MLII', 'II', 'I', 'MLI', 'V5'] 
    valid_lead = ['MLII'] 
    fs_out = 360
    test_ratio = 0.2

    train_ind = []
    test_ind = []
    all_pid = []
    all_data = []
    all_label = []
    all_group = []

    with open(os.path.join(path, 'RECORDS'), 'r') as fin:
        all_record_name = fin.read().strip().split('\n')
    test_pid = random.choices(all_record_name, k=int(len(all_record_name)*test_ratio))
    train_pid = list(set(all_record_name) - set(test_pid))

    for record_name in all_record_name:
        try:
            tmp_ann_res = wfdb.rdann(path + '/' + record_name, 'atr').__dict__
            tmp_data_res = wfdb.rdsamp(path + '/' + record_name)
        except:
            print('read data failed')
            continue
        fs = tmp_data_res[1]['fs']
        ## total 1 second for each
        left_offset = int(1.0*fs / 2)
        right_offset = int(fs) - int(1.0*fs / 2)

        lead_in_data = tmp_data_res[1]['sig_name']
        my_lead_all = []
        for tmp_lead in valid_lead:
            if tmp_lead in lead_in_data:
                my_lead_all.append(tmp_lead)
        if len(my_lead_all) != 0:
            for my_lead in my_lead_all:
                channel = lead_in_data.index(my_lead)
                tmp_data = tmp_data_res[0][:, channel]

                idx_list = list(tmp_ann_res['sample'])
                label_list = tmp_ann_res['symbol']
                for i in range(len(label_list)):
                    s = label_list[i]
                    if s in label_group_map.keys():
                        idx_start = idx_list[i]-left_offset
                        idx_end = idx_list[i]+right_offset
                        if idx_start < 0 or idx_end > len(tmp_data):
                            continue
                        else:
                            all_pid.append(record_name)
                            all_data.append(resample_unequal(tmp_data[idx_start:idx_end], fs, fs_out))
                            all_label.append(s)
                            all_group.append(label_group_map[s])
                            if record_name in train_pid:
                                train_ind.append(True)
                                test_ind.append(False)
                            else:
                                train_ind.append(False)
                                test_ind.append(True)
                    else:
                        continue
                print('record_name:{}, lead:{}, fs:{}, cumcount: {}'.format(record_name, my_lead, fs, len(all_pid)))
        else:
            print('lead in data: [{0}]. no valid lead in {1}'.format(lead_in_data, record_name))
            continue

    all_pid = np.array(all_pid)
    all_data = np.array(all_data)
    all_label = np.array(all_label)
    all_group = np.array(all_group)
    train_ind = np.array(train_ind)
    test_ind = np.array(test_ind)
    print(all_data.shape)
    print(all_label.shape, np.sum(train_ind), np.sum(test_ind))
    print(Counter(all_label))
    print(Counter(all_group))
    print(Counter(all_group[train_ind]), Counter(all_group[test_ind]))
    np.save(os.path.join(save_path, 'mitdb_data.npy'), all_data)
    np.save(os.path.join(save_path, 'mitdb_label.npy'), all_label)
    np.save(os.path.join(save_path, 'mitdb_group.npy'), all_group)
    np.save(os.path.join(save_path, 'mitdb_pid.npy'), all_pid)
    np.save(os.path.join(save_path, 'mitdb_train_ind.npy'), train_ind)
    np.save(os.path.join(save_path, 'mitdb_test_ind.npy'), test_ind)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras import layers, models, callbacks, utils
import keras_nlp as nlp
import keras as keras
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# List all physical devices visible to TensorFlow
devices = tf.config.list_physical_devices()
print("Devices:", devices)

# Specifically check for GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available.")
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("GPU is not available.")

print("TensorFlow version:", tf.__version__)
print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])

def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    """
    figure = plt.figure(figsize=(8, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    return figure

def train_and_evaluate_model(model, train_x, validation_x, train_y, validation_y, class_names, callbacks=None, epochs=20, batch_size=64):
    
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(validation_x, validation_y), callbacks=callbacks)

    pd.DataFrame(history.history).plot(
        figsize=(8, 5), xlim=[0, epochs], ylim=[0, 1], grid=True, xlabel="Epoch",
        style=["r--", "r--.", "b-", "b-*"])
    plt.legend(loc="lower left")
    plt.show()

    # Predictions for confusion matrix and classification report
    Y_pred = model.predict(X_test)
    Y_pred_classes = np.argmax(Y_pred, axis=1)
    Y_true_classes = np.argmax(Y_test, axis=1)

    # Classification report
    print(classification_report(Y_true_classes, Y_pred_classes, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(Y_true_classes, Y_pred_classes)
    fig = plot_confusion_matrix(cm, class_names)
    plt.show()


def label2index(i):
    m = {'N':0, 'S':1, 'V':2, 'F':3, 'Q':4}
    return m[i]


def vgg_block(input, cnn_units):
    output = input
    output = layers.Conv1D(cnn_units, 3, padding='same', activation='relu')(output)
    output = layers.BatchNormalization()(output)
    output = layers.Dropout(0.1)(output)
    output = layers.Conv1D(cnn_units, 3, padding='same', activation='relu')(output)
    output = layers.BatchNormalization()(output)
    output = layers.Dropout(0.1)(output)
    output = layers.Conv1D(cnn_units, 24, padding='same', activation='relu', strides=2)(output)
    output = layers.BatchNormalization()(output)
    output = layers.MaxPooling1D(2, padding='same')(output)
    return output

def create_crt_baseline(number_of_leads=None, cnn_units=128, vgg_blocks=1, rnn_units=64,
                   transformer_encoders=4, att_dim=64, att_heads=8, fnn_units=64, num_classes=5):
    input = layers.Input(shape=(None, number_of_leads))
    output = input

    for _ in range(vgg_blocks):
        output = layers.BatchNormalization()(output)
        output = vgg_block(output, cnn_units)

    output = layers.BatchNormalization()(output)
    output = layers.Bidirectional(layers.GRU(rnn_units, return_sequences=True), merge_mode='sum')(output)
    output = layers.BatchNormalization()(output)

    if transformer_encoders > 0:
        output = output + nlp.layers.SinePositionEncoding(max_wavelength=10000)(output)

        for _ in range(transformer_encoders):
            output = layers.BatchNormalization()(output)
            output = nlp.layers.TransformerEncoder(att_dim, att_heads)(output)

        output = layers.GlobalAveragePooling1D()(output)

    output = layers.Dropout(0.2)(output)
    output = layers.Dense(fnn_units, activation='relu')(output)
    output = layers.BatchNormalization()(output)
    output = layers.Dropout(0.2)(output)
    output = layers.Dense(fnn_units, activation='relu')(output)
    output = layers.BatchNormalization()(output)
    output = layers.Dense(num_classes, activation='softmax')(output)
    model = keras.models.Model(input, output)
    return model

def load_and_preprocess_data(path, num_classes):
    data = np.load(os.path.join(path, 'mitdb_data.npy'))
    label_str = np.load(os.path.join(path, 'mitdb_group.npy'))
    label = np.array([label2index(i) for i in label_str])

    train_ind = np.load(os.path.join(path, 'mitdb_train_ind.npy'))
    test_ind = np.load(os.path.join(path, 'mitdb_test_ind.npy'))
    data = preprocessing.scale(data, axis=1)
    X_train = data[train_ind]
    X_test = data[test_ind]
    Y_train = utils.to_categorical(label[train_ind], num_classes=num_classes)
    Y_test = utils.to_categorical(label[test_ind], num_classes=num_classes)

    ros = RandomOverSampler(random_state=0)
    X_train, Y_train = ros.fit_resample(X_train, Y_train)

    return X_train, X_test, Y_train, Y_test

In [ ]:
import numpy as np
import os
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras import layers, models, callbacks, utils
import keras_nlp as nlp
from tensorflow.keras.layers import LayerNormalization, Dense, Dropout, Add, Conv1D, MaxPooling1D, BatchNormalization, ReLU, Input, MultiHeadAttention
import tensorflow as tf

# Define the VGG block
def vgg_block(input_tensor, filters):
    x = Conv1D(filters, 3, padding='same', activation='relu')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(2, padding='same')(x)
    return x

# Define the Transformer encoder block
def transformer_encoder_block(inputs, att_dim, att_heads, dropout_rate, ff_dim):
    norm_input = LayerNormalization()(inputs)
    attention_output = MultiHeadAttention(num_heads=att_heads, key_dim=att_dim)(norm_input, norm_input)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization()(attention_output)

    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization()(ff_output)

# Define the Bottleneck block
def bottleneck_block(x, in_channels, out_channels, kernel_size, stride, downsample, use_bn, use_do):
    identity = x

    x = Conv1D(filters=out_channels, kernel_size=kernel_size, strides=stride, padding='same')(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = ReLU()(x)
    if use_do:
        x = Dropout(0.5)(x)

    x = Conv1D(filters=out_channels, kernel_size=kernel_size, strides=1, padding='same')(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = ReLU()(x)
    if use_do:
        x = Dropout(0.5)(x)

    if downsample:
        identity = MaxPooling1D(pool_size=stride, padding='same')(identity)

    if out_channels != in_channels:
        identity = Conv1D(filters=out_channels, kernel_size=1, padding='same')(identity)

    x = layers.add([x, identity])
    return x

# Model creation function
def create_crtnet_bottleneck(number_of_leads=1,
                   cnn_units=128,
                   vgg_blocks=1,
                   rnn_units=64,
                   transformer_encoders=4,
                   att_dim=64,
                   att_heads=8,
                   ff_dim=64,
                   dropout_rate=0.1,
                   num_classes=5):
    input = Input(shape=(None, number_of_leads))
    x = input

    for _ in range(vgg_blocks):
        x = vgg_block(x, cnn_units)

    # Add bottleneck blocks
    x = bottleneck_block(x, in_channels=cnn_units, out_channels=cnn_units, kernel_size=3, stride=2, downsample=True, use_bn=True, use_do=True)

    x = layers.Bidirectional(layers.GRU(rnn_units, return_sequences=True), merge_mode='sum')(x)

    for _ in range(transformer_encoders):
        x = transformer_encoder_block(x, att_dim, att_heads, dropout_rate, ff_dim)

    x = layers.GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=input, outputs=x)
    return model

In [ ]:
path = 'data/'
num_classes = 5
class_names = ['N', 'S', 'V', 'F', 'Q']  # Update based on your classes

X_train, X_test, Y_train, Y_test = load_and_preprocess_data(path, num_classes)

model = create_crt_baseline(number_of_leads=1, num_classes=num_classes)
model.summary()

# Define callbacks
stopping = callbacks.EarlyStopping(patience=5)
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.001*0.001)

initial_learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

train_and_evaluate_model(model, X_train, X_test, Y_train, Y_test, class_names=class_names, epochs=20, batch_size=64)

    